In [1]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

In [4]:
api_key = os.getenv('EPA_API_KEY')

In [21]:
def fetch_aqi_data(year, api_key):
    url = 'https://aqs.epa.gov/data/api/annualData/byCBSA'
    params = {
        'email': 'massmediafan@gmail.com',
        'key': api_key,
        'param': '88101',  # Column for PM2.5
        'bdate': f'{year}0101',
        'edate': f'{year}1231',
        'cbsa': cbsa  # CBSA code for Tulsa, LA and NY define later
    }
    response = requests.get(url, params=params)
    data = response.json()
    # Debugging: print the response content
    print(f"Year: {year}, CBSA: {cbsa}, Response: {data}")

    if 'Data' in data:
        return pd.DataFrame(data['Data'])
    else:
        return pd.DataFrame()  # Return an empty DataFrame if 'Data' key is missing

In [24]:
api_key = os.getenv('EPA_API_KEY')

In [25]:
# Range of years
years = range(2014, 2024)  # Adjust this range as needed

In [26]:
# CBSA codes
cbsa_codes = {
    'Tulsa': '35620', 
    'Los Angeles': '31080',
    'New York': '46140'
}

In [27]:
def fetch_aqi_data(year, api_key, cbsa_code):
    # EmptyDataFrame to avoid NoneType errors
    
    return pd.DataFrame()  # Temporary placeholder

# Concatenate data for the years and CBSA codes
all_data = pd.DataFrame()

for cbsa_name, cbsa_code in cbsa_codes.items():
    for year in years:
        yearly_data = fetch_aqi_data(year, api_key, cbsa_code)
        if not yearly_data.empty:
            yearly_data['CBSA'] = cbsa_name  # Add a column for CBSA name
            all_data = pd.concat([all_data, yearly_data])

In [28]:
# Reset the index
all_data.reset_index(drop=True, inplace=True)

In [29]:
# Convert the date column to datetime format if needed
if 'second_max_datetime' in all_data.columns:
    all_data['second_max_datetime'] = pd.to_datetime(all_data['second_max_datetime'])

In [30]:
# Group the data by year and CBSA code and calculate the average first_max_value for each year
if not all_data.empty and 'second_max_datetime' in all_data.columns:
    yearly_aqi = all_data.groupby([all_data['second_max_datetime'].dt.year, 'CBSA'])['first_max_value'].mean().unstack()
    
    # Plot 
    plt.figure(figsize=(10, 6))
    for cbsa in cbsa_codes.keys():
        if cbsa in yearly_aqi.columns:
            plt.plot(yearly_aqi.index, yearly_aqi[cbsa], marker='o', linestyle='-', label=cbsa)

    plt.title('Average AQI (First Max Value) Over the Last Decade')
    plt.xlabel('Year')
    plt.ylabel('Average AQI (First Max Value)')
    plt.legend(title='CBSA')
    plt.grid(True)
    plt.show()
else:
    print("No valid data to plot.")

No valid data to plot.
